# ADS2 - Assignment 1 - Data Handling and Processing with PySpark

**STUDENT NAME -** **CORNELIUS OLUWAGBENGA AKINLOYE**

**STUDENT ID -** **20063102**

In this assignment, you will be analysing the popularity of films and TV shows on the streaming platform, Netflix. Using your knowledge of PySpark DataFrames and Spark SQL, you will produce a number of "downstream" data products to analyse trends in global streaming habits.

Download the dataset from this [Kaggle](https://www.kaggle.com/dhruvildave/netflix-top-10-tv-shows-and-films) page. A copy of the `all_weeks_countries.csv` file is also available on the canvas page for this assignment.

Your task is to load in the data and produce a number of "downstream" data products and plots as described below.

The PySpark installation and setup is provided below for conveinience.

IMPORTANT: DO NOT EDIT OR REMOVE THE COMMENT TAGS IN THE CODE CELLS

In [4]:
# CodeGrade Tag Init1

# Apache Spark uses Java, so first we must install that
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [5]:
# CodeGrade Tag Init2
# Mount Google Drive and unpack Spark
from google.colab import drive
drive.mount('/content/drive')
!wget -q https://dlcdn.apache.org/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz
!tar xzf spark-3.5.0-bin-hadoop3.tgz

Mounted at /content/drive


In [7]:
# CodeGrade Tag Init3
# Set up environment variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "spark-3.5.0-bin-hadoop3"

In [8]:
# CodeGrade Tag Init4
# Install findspark, which helps python locate the pyspark module files
!pip install -q findspark
import findspark
findspark.init()

In [9]:
# Finally, we initialse a "SparkSession", which handles the computations
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [10]:
# Load the all_weeks_countries.csv into your Colab Notebook as a DataFrame.
netflixcsvpath = '/content/drive/My Drive/allweekscountries.csv'

# Data is loaded with header: True and an inferred schema
netflixDF = (spark
           .read
           .option('header', 'True')
           .option('inferSchema', 'True')
           .csv(netflixcsvpath)
          )

In [11]:
# pyspark.sql.functions countains all the transformations and actions you will
# need
from pyspark.sql import functions as F

# Exercise 1 - Data Preparation


1.   Create two separate DataFrames for Films and TV.
2.   For the Films data, drop the column containing the season names.
3.   For the TV data, replace any null values in the season name column with the show name.

In [12]:
# Show the first 5 rows of data and print the schema
netflixDF.show(5)


+------------+------------+----------+--------+-----------+--------------------+------------+--------------------------+
|country_name|country_iso2|      week|category|weekly_rank|          show_title|season_title|cumulative_weeks_in_top_10|
+------------+------------+----------+--------+-----------+--------------------+------------+--------------------------+
|   Argentina|          AR|2022-08-21|   Films|          1|      Look Both Ways|        NULL|                         1|
|   Argentina|          AR|2022-08-21|   Films|          2|           Day Shift|        NULL|                         2|
|   Argentina|          AR|2022-08-21|   Films|          3|Bank Robbers: The...|        NULL|                         2|
|   Argentina|          AR|2022-08-21|   Films|          4|   The Next 365 Days|        NULL|                         1|
|   Argentina|          AR|2022-08-21|   Films|          5|The Angry Birds M...|        NULL|                         1|
+------------+------------+-----

In [13]:
# CodeGrade Tag Ex1a

### Display the table and its schema
netflixDF.printSchema()



root
 |-- country_name: string (nullable = true)
 |-- country_iso2: string (nullable = true)
 |-- week: date (nullable = true)
 |-- category: string (nullable = true)
 |-- weekly_rank: integer (nullable = true)
 |-- show_title: string (nullable = true)
 |-- season_title: string (nullable = true)
 |-- cumulative_weeks_in_top_10: integer (nullable = true)



In [14]:
# CodeGrade Tag Ex1b

### Seperate the data into two DataFrames for Films and TV
### Call the dataframes tvDF and filmsDF
filmsDF = netflixDF.where("category == 'Films'")

tvDF = netflixDF.where("category == 'TV'")


filmsDF.show(5)
tvDF.show(5)


+------------+------------+----------+--------+-----------+--------------------+------------+--------------------------+
|country_name|country_iso2|      week|category|weekly_rank|          show_title|season_title|cumulative_weeks_in_top_10|
+------------+------------+----------+--------+-----------+--------------------+------------+--------------------------+
|   Argentina|          AR|2022-08-21|   Films|          1|      Look Both Ways|        NULL|                         1|
|   Argentina|          AR|2022-08-21|   Films|          2|           Day Shift|        NULL|                         2|
|   Argentina|          AR|2022-08-21|   Films|          3|Bank Robbers: The...|        NULL|                         2|
|   Argentina|          AR|2022-08-21|   Films|          4|   The Next 365 Days|        NULL|                         1|
|   Argentina|          AR|2022-08-21|   Films|          5|The Angry Birds M...|        NULL|                         1|
+------------+------------+-----

In [15]:
# CodeGrade Tag Ex1c
from pyspark.sql.functions import col, coalesce
### Drop the 'season_title' column from the Films DataFrame, display the table
Season_Title_Dropped_filmsDF = filmsDF.drop(col('season_title'))

Season_Title_Dropped_filmsDF.show(5)


+------------+------------+----------+--------+-----------+--------------------+--------------------------+
|country_name|country_iso2|      week|category|weekly_rank|          show_title|cumulative_weeks_in_top_10|
+------------+------------+----------+--------+-----------+--------------------+--------------------------+
|   Argentina|          AR|2022-08-21|   Films|          1|      Look Both Ways|                         1|
|   Argentina|          AR|2022-08-21|   Films|          2|           Day Shift|                         2|
|   Argentina|          AR|2022-08-21|   Films|          3|Bank Robbers: The...|                         2|
|   Argentina|          AR|2022-08-21|   Films|          4|   The Next 365 Days|                         1|
|   Argentina|          AR|2022-08-21|   Films|          5|The Angry Birds M...|                         1|
+------------+------------+----------+--------+-----------+--------------------+--------------------------+
only showing top 5 rows



In [16]:
# CodeGrade Tag Ex1d

### Use the F.isnull function to create a column showing where there are null
### values in the 'season_title' column. Replace the null values with the
### corresponding value from the 'show_title' column, then replace the
### 'season_title' column in the tvDF DataFrame.
FnullDF = tvDF.withColumn("IsNull_season_title", col("season_title").isNull())
FnullDF.show()
# Replace null values in 'Column2' with values from 'Column1'
tvDF = tvDF.withColumn("season_title", coalesce(col("season_title"), col("show_title")))

tvDF = tvDF.withColumn("show_title", coalesce(col("show_title"), col("season_title")))

tvDF.show()

+------------+------------+----------+--------+-----------+--------------------+--------------------+--------------------------+-------------------+
|country_name|country_iso2|      week|category|weekly_rank|          show_title|        season_title|cumulative_weeks_in_top_10|IsNull_season_title|
+------------+------------+----------+--------+-----------+--------------------+--------------------+--------------------------+-------------------+
|   Argentina|          AR|2022-08-21|      TV|          1| Pasión de Gavilanes|Pasión de Gavilan...|                         5|              false|
|   Argentina|          AR|2022-08-21|      TV|          2|        Another Self|Another Self: Sea...|                         4|              false|
|   Argentina|          AR|2022-08-21|      TV|          3| Pasión de Gavilanes|Pasión de Gavilan...|                        58|              false|
|   Argentina|          AR|2022-08-21|      TV|          4|            Manifest|  Manifest: Season 1|     

# Exercise 2

Making use of the "groupBy" and "where" methods, find the number of weeks the show "Stranger Things" was in the Top 10 for the United Kingdom across all seasons. Store your result in a variable named "STWeeks"


In [17]:
# CodeGrade Tag Ex2
### Use the "where" method to create a new dataframe containing the data for
### the show Stranger Things in the Uniter Kingdom. Call this dataframe STDF.
STDF = netflixDF.where("show_title == 'Stranger Things' AND country_name == 'United Kingdom'")
print(STDF.count())
STDF.show()
### Using "groupBy" method and "F.count_distinct" function, find the total number of weeks
### Stranger Things spent in the top 10 of the UK, across all seasons. Show the
### result.
result = STDF.groupBy("season_title").agg(F.countDistinct("cumulative_weeks_in_top_10").alias("TotalWeeksInTop10"))
result.show()


46
+--------------+------------+----------+--------+-----------+---------------+-----------------+--------------------------+
|  country_name|country_iso2|      week|category|weekly_rank|     show_title|     season_title|cumulative_weeks_in_top_10|
+--------------+------------+----------+--------+-----------+---------------+-----------------+--------------------------+
|United Kingdom|          GB|2022-08-21|      TV|          3|Stranger Things|Stranger Things 4|                        13|
|United Kingdom|          GB|2022-08-14|      TV|          3|Stranger Things|Stranger Things 4|                        12|
|United Kingdom|          GB|2022-08-14|      TV|         10|Stranger Things|Stranger Things 3|                        12|
|United Kingdom|          GB|2022-08-07|      TV|          2|Stranger Things|Stranger Things 4|                        11|
|United Kingdom|          GB|2022-08-07|      TV|          9|Stranger Things|Stranger Things 3|                        11|
|United Kingd

# Exercise 3

Produce a dataframe containing only the Top 25 TV seasons in the UK, based on the number of weeks they spent in the Top 10.

In [18]:
# CodeGrade Tag Ex3
### Produce a dataframe containing the top 25 seasons by number of weeks in the
### top 10 of the United Kingdom, sorted by number of weeks. Store the dataframe
### in a variable called Top25
Top25 = netflixDF.where("country_name == 'United Kingdom'").orderBy('cumulative_weeks_in_top_10', ascending=False).limit(25)
Top25.show(30);

+--------------+------------+----------+--------+-----------+----------------+--------------------+--------------------------+
|  country_name|country_iso2|      week|category|weekly_rank|      show_title|        season_title|cumulative_weeks_in_top_10|
+--------------+------------+----------+--------+-----------+----------------+--------------------+--------------------------+
|United Kingdom|          GB|2022-08-21|      TV|          3| Stranger Things|   Stranger Things 4|                        13|
|United Kingdom|          GB|2022-08-14|      TV|          3| Stranger Things|   Stranger Things 4|                        12|
|United Kingdom|          GB|2022-08-14|      TV|         10| Stranger Things|   Stranger Things 3|                        12|
|United Kingdom|          GB|2022-06-05|      TV|          7|           Ozark|     Ozark: Season 4|                        12|
|United Kingdom|          GB|2022-08-07|      TV|          2| Stranger Things|   Stranger Things 4|            

# Exercise 4

For the show "Young Sheldon", find the country where each season spent the most time in the Top 10.

In [19]:
# CodeGrade Tag Ex4
### For each season of the show "Young Sheldon" find the countries where it spent
### the most time in the Top 10
ysDF = netflixDF.where("show_title == 'Young Sheldon'")
result = ysDF.groupBy("country_name", "cumulative_weeks_in_top_10").count()
maximum_appearance = result.groupBy("country_name").agg(F.max("cumulative_weeks_in_top_10").alias("MaxAppearances"))
maximum_appearance.show()


+--------------+--------------+
|  country_name|MaxAppearances|
+--------------+--------------+
|         Italy|             3|
|       Ireland|             6|
|        Canada|             6|
|   New Zealand|             3|
|     Australia|             4|
|United Kingdom|             6|
+--------------+--------------+



# Exercise 5

For each country, find the film that spent the most time in the Top 10.

In [25]:
# CodeGrade Tag Ex5
### For each country, find the film that spent the most time in the Top 10
### Display the results in a Dataframe ordered by country name.
from pyspark.sql.window import Window

allCountriesDF = netflixDF.groupBy("country_name")
country_spec = Window.partitionBy("country_name").orderBy(F.desc("cumulative_weeks_in_top_10"))
ranked_df = netflixDF.withColumn("rank", F.rank().over(country_spec))
maximum_appearance = ranked_df.filter("rank = 1").drop("rank")
maximum_appearance.orderBy("country_name", ascending=True).show(50)

+------------------+------------+----------+--------+-----------+--------------------+--------------------+--------------------------+
|      country_name|country_iso2|      week|category|weekly_rank|          show_title|        season_title|cumulative_weeks_in_top_10|
+------------------+------------+----------+--------+-----------+--------------------+--------------------+--------------------------+
|         Argentina|          AR|2022-08-21|      TV|          3| Pasión de Gavilanes|Pasión de Gavilan...|                        58|
|         Australia|          AU|2022-08-21|      TV|          7|     Stranger Things|   Stranger Things 4|                        13|
|           Austria|          AT|2022-07-10|      TV|         10|     The Good Doctor|The Good Doctor: ...|                        14|
|           Bahamas|          BS|2022-08-14|      TV|         10|SpongeBob SquareP...|SpongeBob SquareP...|                        33|
|           Bahrain|          BH|2022-01-02|      TV|  

# Exercise 6

Calculate the number of weeks each film spent at the number 1 spot of each country's Top 10 list. Then find the films that spent the most time in the number 1 spot for each country.

In [27]:
# CodeGrade Tag Ex6a

### Create a column using the F.when function to calculate the number of weeks a
### films spens in the number 1 spot of the Top 10. Use the .otherwise method to
### set rows with no number 1 spots to zero. Use the .alias metod to call this
### column "weeks_at_1"

week_spent_on_1_spec = Window.partitionBy("cumulative_weeks_in_top_10").orderBy(F.desc("cumulative_weeks_in_top_10"))

# Use the rank function to assign a rank to each film within each week
ranked_df = netflixDF.withColumn("rank", F.rank().over(week_spent_on_1_spec))

# Create a new column "weeks_at_1" using F.when
TimeSpentOn1DF = ranked_df.withColumn("weeks_at_1", F.when(F.col("rank") == 1, 1).otherwise(0))

TimeSpentOn1DF.show(50)


+------------+------------+----------+--------+-----------+--------------------+--------------------+--------------------------+----+----------+
|country_name|country_iso2|      week|category|weekly_rank|          show_title|        season_title|cumulative_weeks_in_top_10|rank|weeks_at_1|
+------------+------------+----------+--------+-----------+--------------------+--------------------+--------------------------+----+----------+
|   Argentina|          AR|2022-08-21|   Films|          1|      Look Both Ways|                NULL|                         1|   1|         1|
|   Argentina|          AR|2022-08-21|   Films|          4|   The Next 365 Days|                NULL|                         1|   1|         1|
|   Argentina|          AR|2022-08-21|   Films|          5|The Angry Birds M...|                NULL|                         1|   1|         1|
|   Argentina|          AR|2022-08-21|   Films|          7|      River Runs Red|                NULL|                         1|  

In [29]:
# CodeGrade Tag Ex6b

### Group by country name and show title, and use the .agg method and your new
### column to find the number of weeks each film spent in the top spot for each
### country.
window_spec = Window.partitionBy("cumulative_weeks_in_top_10").orderBy(F.desc("cumulative_weeks_in_top_10"))

# Use the rank function to assign a rank to each film within each week
ranked_df = netflixDF.withColumn("rank", F.rank().over(window_spec))

# Create a new column "weeks_at_1" using F.when
newnetflixDF = netflixDF
newnetflixDF = ranked_df.withColumn("weeks_at_1", F.when(F.col("rank") == 1, 1).otherwise(0))

# Group by country and film, and find the number of weeks each film spent in the top spot
result = newnetflixDF.groupBy("country_name", "show_title").agg(F.sum("weeks_at_1").alias("NumberOfWeeksAt1"))

result.show(5)


+------------+--------------------+----------------+
|country_name|          show_title|NumberOfWeeksAt1|
+------------+--------------------+----------------+
|   Australia|           Bloodshot|               2|
|   Australia|             Traffik|               1|
|   Australia|   Godzilla vs. Kong|               2|
|     Austria|            For Life|              11|
|     Austria|Demon Slayer: Kim...|               3|
+------------+--------------------+----------------+
only showing top 5 rows



In [31]:
# CodeGrade Tag Ex6c

### Produce a dataframe grouped by country name that contains the show title and
### number of weeks at the number 1 spot of the top performing film in each
### country.
# Define a window specification to rank films within each week
country_and_week_spec = Window.partitionBy("country_name", "cumulative_weeks_in_top_10").orderBy(F.desc("cumulative_weeks_in_top_10"))

newnetflixDF = netflixDF

# Use the rank function to assign a rank to each film within each week
ranked_df = newnetflixDF.withColumn("rank", F.rank().over(window_spec))

# Create a new column "weeks_at_1" using F.when
df = ranked_df.withColumn("weeks_at_1", F.when(F.col("rank") == 1, 1).otherwise(0))

# Group by country and film, and find the number of weeks each film spent in the top spot
result = df.groupBy("country_name", "show_title").agg(F.sum("weeks_at_1").alias("NumberOfWeeksAt1"))

# Filter for the top-performing film in each country
top_performersDF = result.withColumn("rank", F.row_number().over(Window.partitionBy("country_name").orderBy(F.desc("NumberOfWeeksAt1"))))
top_performersDF = top_performersDF.filter(F.col("rank") == 1).drop("rank")

top_performersDF.show()

+------------------+--------------------+----------------+
|      country_name|          show_title|NumberOfWeeksAt1|
+------------------+--------------------+----------------+
|         Argentina| Pasión de Gavilanes|              63|
|         Australia|     Stranger Things|              41|
|           Austria|     Stranger Things|              40|
|           Bahamas|SpongeBob SquareP...|              37|
|           Bahrain|     Stranger Things|              36|
|        Bangladesh|     Stranger Things|              44|
|           Belgium|     Stranger Things|              41|
|           Bolivia| Pasión de Gavilanes|              62|
|            Brazil|         Chiquititas|              51|
|          Bulgaria|     Stranger Things|              46|
|            Canada|     Stranger Things|              37|
|             Chile| Pasión de Gavilanes|              55|
|          Colombia|Pablo Escobar, el...|              60|
|        Costa Rica| Pasión de Gavilanes|              6